In [1]:
%display latex

The modified update rule does not use a negative for the other indices.
$$
  x_i = 
  \begin{cases}
    \frac{1}{x_i}, & \text{ if } i = \ell, \\
    \frac{x_i}{x_\ell}, & \text{ otherwise.} \\
  \end{cases}
$$

In [15]:
def frac(x):
    return x - floor(RR(x))

def pivot(xs, *indices):
    coeffs = []
    for l in indices:
        a = [0] * len(xs)
        ys = [0] * len(xs)
        xl = xs[l]
        assert xl != 0, "xl cannot be zero"
        for i, xi in enumerate(xs):
            if i == l:
                a[i] = floor(RR(1 / xi))
                ys[i] = frac(1 / xi)
            else:
                a[i] = floor(RR(xi / xl))
                ys[i] = frac(xi / xl)
        xs = tuple(ys)
        coeffs.append(tuple(a))
    return xs, coeffs

def argmin(xs):
    j = None
    for i in range(len(xs)):
        if j == None or xs[i] < xs[j]:
            j = i
    return j

# Equal Distance between Neighbors

The first polynomial is the "neighbor" polynomial, which is derived from the following equation:
$$
  x_1 = \frac{x_2}{x_1} - 1 = \dots = \frac{x_d}{x_{d-1}} - 1 = \frac{1}{x_d} - 1.
$$
This results in the equation
$$
  x_d^{d+1} + x_d - 1 = 0, \quad x_i = x_d^{d + 1 - i}
$$

In [2]:
def neighbor_distance(xs):
    dist = []
    for x, y in zip([*xs, 1], [1, *xs]):
        if y == 0 or x == 0:
            dist.append(1)
        else:
            dist.append(frac(x / y))
    return dist

def distance_matrix(xs):
    D = [[0 for _ in xs] for _ in xs]
    for i, x in enumerate(xs):
        for j, y in enumerate(xs):
            if x == 0 or y == 0:
                D[i][j] = 1
            elif i == j:
                D[i][j] = frac(1 / x)
            else:
                D[i][j] = frac(x / y)
    return D

# Indices are zero-based. An index of -1 indicates selecting nothing in the first iteration and selecting 0 in the next iteration.
# Similarly, d - 1 indicates selecting d - 1 in the first iteration and nothing in the next iteration.
def select(xs):
    min_index = None
    min_dist = 1
    for i, dist in enumerate(neighbor_distance(xs)):
        if min_dist > dist:
            min_dist = dist
            min_index = i - 1
    return min_index

In [3]:
def neighbor_poly(d):
    return x ** (d+1) + x - 1
neighbor_poly(5)

x^6 + x - 1

The polynomial gives us the field $\mathbb{Q}/ (x^{d+1} + x - 1)$.

In [5]:
p = x^3 - 2
d = p.degree(x)
K.<psi> = NumberField(p, embedding=RR(0.5))
K

Number Field in psi with defining polynomial x^3 - 2 with psi = 1.259921049894873?

For our input, we choose $x_i = \psi^{d+1-i}$.

In [6]:
xs = [psi^(i+1) for i in range(d)]

table([(i, x,RR(x)) for i, x in enumerate(xs)], header_row=[r'$i$', r'$x_i$'])

$i$   $x_i$
├─────┼───────┤
  0     psi
  1     psi^2
  2     2

The distance between neighbors is indeed the same.

In [7]:
dist = distance_matrix(xs)

header = [''] + [rf'$x_{{{i}}}$' for i in range(len(dist))]
table(dist, header_row=header[1:], header_column=header)

,\(x_{0}\),\(x_{1}\),\(x_{2}\)
\(x_{0}\),\(\frac{1}{2} \psi^{2}\),\(\frac{1}{2} \psi^{2}\),\(\frac{1}{2} \psi\)
\(x_{1}\),\(\psi - 1\),\(\frac{1}{2} \psi\),\(\frac{1}{2} \psi^{2}\)
\(x_{2}\),\(\psi^{2} - 1\),\(\psi - 1\),\(\frac{1}{2}\)


Therefore, we can expect a decrease of at least $\psi^3$ over two iterations. However, this condition does not hold up after those two iterations when choosing any element to pivot with. It only holds up if we choose the maximum $x_d$ of those elements. Choosing any other pivot, destroys this solution.

In [8]:
n = 50
d = len(xs)

ys = xs
rows = [[
    *[f'$x_{i+1}$' for i in range(d)], 
    *[fr'$\{{x_{{{i+2}}}/x_{{{i+1}}}\}}$' for i in range(-1, d)], 
    r'$\ell_1$', 
    r'$\ell_2$',
]]

for i in range(n):
    # select pivot
    l = select(ys)
    
    # Add row to the output
    dist   = [round(RR(d), 3) for d in neighbor_distance(ys)]
    values = [round(RR(y), 3) for y in ys]
    rows.append([*values, *dist, '-' if l == -1 else l, '-' if l == d - 1 else l + 1])

    # pivot
    if l == -1:
        ys = pivot(ys, 0)
    elif l == d - 1:
        ys = pivot(ys, d - 1)
    else:
        ys = pivot(ys, l, l + 1)
    
table(rows, header_row=True, align='center')

TypeError: unsupported operand parent(s) for /: '<class 'tuple'>' and 'Integer Ring'

## Other polynomials

$$
    x^{d+1} + \sum_{k=1}^d k x^{d-k} = 1
$$

In [9]:
xs = (1, 4^(1/3), 16^(1/3))
indices = [1, 2] + [2]*20
table([[RR(x) for x in pivot(xs, *indices[:i+1])] for i in range(len(indices))])

TypeError: unable to convert '(1/4*4^(2/3),1/4*4^(2/3),1/2*4^(2/3)*2^(1/3)-1)' to a real number

# $d$-bonacci Numbers


In [145]:
def nbonacci_poly(d):
    i = var('i')
    return x^(d+1) + sum(x^i, i, 1, d) - 1
nbonacci_poly(3), *[r[0] for r in nbonacci_poly(1).roots()]

(x^4 + x^3 + x^2 + x - 1, -1/2*sqrt(5) - 1/2, 1/2*sqrt(5) - 1/2)

In [242]:
y = polygen(RR, 'y')
d = 5

p = nbonacci_poly(d)
psi_real = find_root(p, 0, 1)
K.<psi> = NumberField(p, embedding=RR(psi_real))

In [168]:
xs = tuple(sum(psi^i for i in range(1, k+1)) for k in range(1, d + 1))
xs

(psi,
 psi^2 + psi,
 psi^3 + psi^2 + psi,
 psi^4 + psi^3 + psi^2 + psi,
 psi^5 + psi^4 + psi^3 + psi^2 + psi)

In [216]:
ys = pivot(xs, 0)
argmin(ys), ys

(1,
 (psi^5 + psi^4 + psi^3 + psi^2 + psi,
  psi,
  psi^2 + psi,
  psi^3 + psi^2 + psi,
  psi^4 + psi^3 + psi^2 + psi))

In [254]:
q = factor(-x^6 * p(x=1/x))
phi_real = find_root(q, 1, 2)
L.<phi> = NumberField(q, embedding=RR(phi_real))
q.list()

[-1, -1, -1, -1, -1, -1, 1]

# Worst-Case Analysis

In [60]:
d = 3
xs = tuple(i / (d + 1) for i in range(1, d + 1))
pivot(xs, 0)

(0, 0, 0)

In [133]:
def unpivot(y, values, indices):
    for a, l in zip(values, indices):
        x = [0] * len(ys)
        for i, (yi, ai) in enumerate(zip(y, a)):
            if i == l:
                x[i] = frac(1 / (ai + yi))
            else:
                x[i] = frac((ai + yi) / (a[l] + y[l]))
        y = x
    return y

In [165]:
ys = tuple(sorted(unpivot(xs, [(2, 1, 1)] * 5, [0] * 5)))
table(distance_matrix(ys), header_row=ys, header_column=['', *ys])

,\(\frac{128}{309}\),\(\frac{218}{309}\),\(\frac{73}{103}\)
\(\frac{128}{309}\),\(\frac{53}{128}\),\(\frac{64}{109}\),\(\frac{128}{219}\)
\(\frac{218}{309}\),\(\frac{45}{64}\),\(\frac{91}{218}\),\(\frac{218}{219}\)
\(\frac{73}{103}\),\(\frac{91}{128}\),\(\frac{1}{218}\),\(\frac{30}{73}\)


In [163]:
pivot(ys, 0, 1, 2)

(0, 0, 0)

In [16]:
K.<phi> = NumberField(x^3 - x - 1, embedding=RR(2))
psi = 1/phi

A = matrix([[1, phi, phi^2], [1, -psi]]).transpose()
b = vector([1, 1])
xs = A.solve_right(b)
pivot(xs, 1, 1, *[1]*10)

(0, phi - 1)

In [77]:
def min_pivot(xs, n):
    coeffs = []
    for _ in range(n):
        l = None
        for i, xi in enumerate(xs):
            if (l is None or frac(xs[l]) > frac(xi)) and frac(xi) != 0:
                l = i
        if l is None:
            break
            
        xs, c = pivot(xs, l)
        coeffs += c
    return xs, coeffs

# Brute-Force Search

In [16]:
def brute_force(xs, max_depth, prev=[]):
    if max_depth == 0:
        return []

    # Try single pivot first to make list as short as possible
    for l, xl in enumerate(xs):
        if frac(xl) != 0:
            ys, coeffs = pivot(xs, l)
            if ys in prev:
                return [l]
                
    for l, xl in enumerate(xs):
        if frac(xl) != 0:
            ys, coeffs = pivot(xs, l)
            result = brute_force(ys, max_depth - 1, prev + [xs])
            if result:
                return [l] + result
    return []

In [26]:
d = 3
p = x^d - 13
K.<psi> = NumberField(p, embedding=RR(1))

xs = tuple(psi^i for i in range(d))
indices = brute_force(xs, 20)
xs, indices[:-1]

((1, psi, psi^2), [1, 0, 0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 2, 0, 2, 0, 2, 0])

In [19]:
rows = []
for i in range(len(indices) + 1):
    ys, coeffs = pivot(xs, *indices[:i])
    coeffs.insert(0, (0, 0, 0))
    rows.append((i, *ys, *coeffs[-1]))
table(rows, header_row=['$i$', '$x_1$', '$x_2$', '$x_3$'])

$i$   $x_1$                        $x_2$       $x_3$
├─────┼────────────────────────────┼───────────┼───────────────────────────┤
  0     1                            psi         psi^2
  1     1/4*psi^2                    1/4*psi^2   psi - 1
  2     psi - 1                      0           psi^2 - psi
  3     1/3*psi^2 + 1/3*psi - 2/3    0           psi - 1
  4     1/3*psi - 1/3                0           1/3*psi^2 + 1/3*psi - 2/3
  5     psi^2 + psi - 4              0           psi - 1
  6     -2/3*psi^2 + 1/3*psi + 4/3   0           1/3*psi^2 + 1/3*psi - 2/3
  7     1/2*psi^2 - 1                0           1/4*psi^2 + 1/2*psi - 1
  8     -1/5*psi^2 + 1/5*psi + 4/5   0           2/5*psi^2 + 3/5*psi - 8/5
  9     1/4*psi^2                    0           psi - 1
  10    psi - 1                      0           psi^2 - psi

In [130]:
ys = xs
index = {ys: 0}
for i, l in enumerate(indices):
    ys = pivot(ys, l)[0]
    if ys in index:
        period_start, period_end = index[ys], i + 1
        break
    index[ys] = i + 1
period_start, period_end

(2, 8)

Why are the alternatives worse?

In [133]:
ys = xs
rows = []
for i, l in enumerate(indices):
    row = [l, tuple(RR(y) for y in ys)]
    for k in range(len(ys)):
        if frac(ys[k]) != 0:
            _, zs = pivot(ys, k)
            row += zs
        else:
            row.append((0, 0, 0))
    ys, _ = pivot(ys, l)
    rows.append(row)
table(rows, header_row=[r'$\ell$', '$x$', '$x^{(0)}$', '$x^{(1)}$', '$x^{(2)}'])

\(\ell\),\(x\),\(x^{(0)}\),\(x^{(1)}\),\(x^{(2)}\)
\(0\),"\(\left(3.65930571002297, 1.00000000000000, 1.91293118277239\right)\)","\(\left(0, 0, 0\right)\)","\(\left(0, 0, 0\right)\)","\(\left(1, 0, 0\right)\)"
\(0\),"\(\left(0.273275883253198, 0.273275883253198, 0.522757958574710\right)\)","\(\left(3, 1, 1\right)\)","\(\left(1, 3, 1\right)\)","\(\left(0, 0, 1\right)\)"
\(2\),"\(\left(0.659305710022972, 0.000000000000000, 0.912931182772389\right)\)","\(\left(1, 0, 1\right)\)","\(\left(0, 0, 0\right)\)","\(\left(0, 0, 1\right)\)"
\(0\),"\(\left(0.722185551840602, 0.000000000000000, 0.0953728154658935\right)\)","\(\left(1, 0, 0\right)\)","\(\left(0, 0, 0\right)\)","\(\left(7, 0, 10\right)\)"
\(2\),"\(\left(0.384685691165303, 0.000000000000000, 0.132061372901772\right)\)","\(\left(2, 0, 0\right)\)","\(\left(0, 0, 0\right)\)","\(\left(2, 0, 7\right)\)"
\(2\),"\(\left(0.912931182772389, 0.000000000000000, 0.572236892795360\right)\)","\(\left(1, 0, 0\right)\)","\(\left(0, 0, 0\right)\)","\(\left(1, 0, 1\right)\)"
\(0\),"\(\left(0.595372815465893, 0.000000000000000, 0.747528012594625\right)\)","\(\left(1, 0, 1\right)\)","\(\left(0, 0, 0\right)\)","\(\left(0, 0, 1\right)\)"
\(2\),"\(\left(0.679619851668028, 0.000000000000000, 0.255562889631880\right)\)","\(\left(1, 0, 0\right)\)","\(\left(0, 0, 0\right)\)","\(\left(2, 0, 3\right)\)"
